In [23]:
# Topic Modeling Research Dashboard


In [38]:
#  Install Required Libraries (run once)
!pip install numpy pandas scikit-learn matplotlib seaborn pyLDAvis gensim nltk spacy umap-learn bertopic

In [25]:
#  Imports

import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation, NMF

import gensim
from gensim.corpora import Dictionary
from gensim.models import CoherenceModel

import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

from bertopic import BERTopic


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [26]:
# Load Dataset
data = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))
documents = data.data
print(f"Total documents: {len(documents)}")


Total documents: 18846


In [27]:
#  Preprocessing

stop_words = stopwords.words('english')

def preprocess(text):
    text = str(text).lower()
    text = re.sub(r'\W+', ' ', text)
    return text

documents_clean = [preprocess(doc) for doc in documents]

In [28]:
# Tokenize for gensim coherence later
tokenized_docs = [doc.split() for doc in documents_clean]

In [29]:
# Create Dictionary and Corpus for Gensim

dictionary = Dictionary(tokenized_docs)
corpus = [dictionary.doc2bow(text) for text in tokenized_docs]


In [30]:
#  LDA using Gensim (recommended for pyLDAvis)

num_topics = 10
lda_gensim = gensim.models.LdaModel(corpus=corpus,
                                   id2word=dictionary,
                                   num_topics=num_topics,
                                   random_state=42,
                                   passes=10,
                                   alpha='auto',
                                   per_word_topics=True)

In [31]:

#  Evaluate LDA

coherence_lda = CoherenceModel(model=lda_gensim, texts=tokenized_docs, dictionary=dictionary, coherence='c_v').get_coherence()
print(f"LDA Coherence Score: {coherence_lda:.4f}")


LDA Coherence Score: 0.5626


In [32]:
# Visualize LDA with pyLDAvis

panel = gensimvis.prepare(lda_gensim, corpus, dictionary)
panel  # In Jupyter, this is interactive

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7     -0.279579 -0.059523       1        1  28.128025
2     -0.266755 -0.024828       2        1  23.940388
5     -0.232062 -0.000300       3        1  13.901233
9     -0.186526 -0.045777       4        1  11.425620
3     -0.180472  0.001242       5        1   9.040188
0      0.151666  0.337023       6        1   8.224496
1      0.169866  0.220441       7        1   4.031090
8      0.241458 -0.090549       8        1   1.235130
6      0.290769 -0.171495       9        1   0.061408
4      0.291634 -0.166235      10        1   0.012423, topic_info=         Term           Freq          Total Category  logprob  loglift
27230      ax   53627.000000   53627.000000  Default  30.0000  30.0000
76        the  177148.000000  177148.000000  Default  29.0000  29.0000
294         0   14919.000000   14919.000000  Default  28.0000  28.0000
89          1   16193.000000   16193.000000  Default  27.0000  27.0000
493         m   17025.000000   17025.000000  Default  26.0000  26.0000
...       ...            ...            ...      ...      ...      ...
17887     231       0.345874      81.614178  Topic10  -7.2239   3.5297
16278  rainer       0.141004      13.206526  Topic10  -8.1212   4.4537
2341       49       0.298290     266.396471  Topic10  -7.3719   2.1987
7942   inform       0.150336      61.356115  Topic10  -8.0571   2.9818
14024    univ       0.108217      32.835335  Topic10  -8.3859   3.2783

[760 rows x 6 columns], token_table=      Topic      Freq  Term
term                       
294       3  0.002949     0
294       5  0.067762     0
294       6  0.218098     0
294       7  0.711132     0
390       1  0.018694    00
...     ...       ...   ...
244       3  0.113128  your
244       5  0.140893  your
1946      5  0.126111     z
1946      6  0.871773     z
1946      8  0.001975     z

[1658 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 3, 6, 10, 4, 1, 2, 9, 7, 5])

In [33]:
#  NMF using TF-IDF

tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, stop_words=stop_words)
tfidf_matrix = tfidf_vectorizer.fit_transform(documents_clean)

nmf_model = NMF(n_components=num_topics, random_state=42)
nmf_model.fit(tfidf_matrix)

# Display top words per topic for NMF
def display_topics(model, feature_names, num_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print(f"Topic {topic_idx+1}: ", " | ".join([feature_names[i] for i in topic.argsort()[:-num_top_words - 1:-1]]))

num_top_words = 10
display_topics(nmf_model, tfidf_vectorizer.get_feature_names_out(), num_top_words)


Topic 1:  would | one | like | think | get | good | time | much | well | know
Topic 2:  thanks | please | anyone | mail | know | advance | hi | looking | email | info
Topic 3:  god | jesus | bible | believe | christ | faith | christian | christians | sin | us
Topic 4:  drive | scsi | ide | card | disk | controller | hard | drives | bus | floppy
Topic 5:  game | games | team | year | hockey | baseball | last | season | play | players
Topic 6:  windows | dos | file | program | files | window | use | using | version | run
Topic 7:  00 | 10 | new | sale | price | 50 | 20 | shipping | 15 | offer
Topic 8:  key | chip | encryption | clipper | keys | government | escrow | system | algorithm | use
Topic 9:  edu | geb | dsl | cadre | n3jxp | chastity | pitt | skepticism | intellect | shameful
Topic 10:  people | government | israel | armenian | jews | armenians | gun | state | rights | children


In [34]:
#  BERTopic (Transformer-based)

topic_model = BERTopic()
topics, probs = topic_model.fit_transform(documents_clean)

# Visualize BERTopic topics
topic_model.visualize_topics()



Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
